# Random_forest-Company_data

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv('Company_Data.csv')
df.head(10)

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
5,10.81,124,113,13,501,72,Bad,78,16,No,Yes
6,6.63,115,105,0,45,108,Medium,71,15,Yes,No
7,11.85,136,81,15,425,120,Good,67,10,Yes,Yes
8,6.54,132,110,0,108,124,Medium,76,10,No,No
9,4.69,132,113,0,131,124,Medium,76,17,No,Yes


In [3]:
df.shape

(400, 11)

In [4]:
df.loc[df.Sales>=8.5,'sales']='high'
df.loc[df.Sales<8.5,'sales']='low'
df.drop('Sales',1,inplace=True)
df

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,138,73,11,276,120,Bad,42,17,Yes,Yes,high
1,111,48,16,260,83,Good,65,10,Yes,Yes,high
2,113,35,10,269,80,Medium,59,12,Yes,Yes,high
3,117,100,4,466,97,Medium,55,14,Yes,Yes,low
4,141,64,3,340,128,Bad,38,13,Yes,No,low
...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,Good,33,14,Yes,Yes,high
396,139,23,3,37,120,Medium,55,11,No,Yes,low
397,162,26,12,368,159,Medium,40,18,Yes,Yes,low
398,100,79,7,284,95,Bad,50,12,Yes,Yes,low


In [5]:
df.shape

(400, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CompPrice    400 non-null    int64 
 1   Income       400 non-null    int64 
 2   Advertising  400 non-null    int64 
 3   Population   400 non-null    int64 
 4   Price        400 non-null    int64 
 5   ShelveLoc    400 non-null    object
 6   Age          400 non-null    int64 
 7   Education    400 non-null    int64 
 8   Urban        400 non-null    object
 9   US           400 non-null    object
 10  sales        400 non-null    object
dtypes: int64(7), object(4)
memory usage: 34.5+ KB


In [8]:
df.describe()

,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
df.iloc[:,8]=labelencoder.fit_transform(df.iloc[:,8])
df.iloc[:,9]=labelencoder.fit_transform(df.iloc[:,9])
df.iloc[:,5]=labelencoder.fit_transform(df.iloc[:,5])
df

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales
0,138,73,11,276,120,0,42,17,1,1,high
1,111,48,16,260,83,1,65,10,1,1,high
2,113,35,10,269,80,2,59,12,1,1,high
3,117,100,4,466,97,2,55,14,1,1,low
4,141,64,3,340,128,0,38,13,1,0,low
...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1,high
396,139,23,3,37,120,2,55,11,0,1,low
397,162,26,12,368,159,2,40,18,1,1,low
398,100,79,7,284,95,0,50,12,1,1,low


In [12]:
colname=list(df.columns)
colname

['CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'ShelveLoc',
 'Age',
 'Education',
 'Urban',
 'US',
 'sales']

In [13]:
x=colname[0:10]
y=colname[10]

In [14]:
x

['CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'ShelveLoc',
 'Age',
 'Education',
 'Urban',
 'US']

In [15]:
y

'sales'

In [18]:
x1=df[x]
y1=df[y]

In [22]:
random=RandomForestClassifier(n_jobs=2,oob_score=True,n_estimators=1000,criterion="entropy")
random.fit(x1,y1)

RandomForestClassifier(criterion='entropy', n_estimators=1000, n_jobs=2,
                       oob_score=True)

In [23]:
df['prediction']=random.predict(x1)
df

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,sales,prediction
0,138,73,11,276,120,0,42,17,1,1,high,high
1,111,48,16,260,83,1,65,10,1,1,high,high
2,113,35,10,269,80,2,59,12,1,1,high,high
3,117,100,4,466,97,2,55,14,1,1,low,low
4,141,64,3,340,128,0,38,13,1,0,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1,high,high
396,139,23,3,37,120,2,55,11,0,1,low,low
397,162,26,12,368,159,2,40,18,1,1,low,low
398,100,79,7,284,95,0,50,12,1,1,low,low


In [24]:
confusion_matrix(df['sales'],df['prediction'])


array([[143,   0],
       [  0, 257]], dtype=int64)

In [25]:
# Bagging Classifier

from sklearn.ensemble import BaggingClassifier
seed=3
kfold=KFold(n_splits=10,random_state=seed,shuffle=True)
cart=DecisionTreeClassifier()
num_trees=10
model=BaggingClassifier(base_estimator=cart,n_estimators=num_trees,random_state=seed)
results=cross_val_score(model,x1,y1,cv=kfold)
print(results.mean())

0.7849999999999999


In [27]:
# AdaBoost Classifier

from sklearn.ensemble import AdaBoostClassifier
num_trees=10
seed=7
kfold=KFold(n_splits=10,random_state=seed,shuffle=True)
model1=AdaBoostClassifier(n_estimators=num_trees,random_state=seed)
results1=cross_val_score(model1,x1,y1,cv=kfold)
print(results1.mean())

0.8225
